###### General Steps to Follow
1. Importing Packages
1. Criteria 1 - Angles
1. Criteria 2 - Angles
1. Criteria 3 - Angles
1. Criteria 4 - Angles
1. Full Test

# -------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, TimeDistributed, Input, Reshape, Flatten, GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

2024-04-14 13:01:59.884889: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 13:02:00.641523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# -------------------------------------------------------------------------------------------------------------

In [2]:
def evaluate(output, y_true):
    y_pred = np.zeros((output.shape[0],1), dtype = "int")
    y_pred[(output > 0.5)] = 1
    y_pred = y_pred.reshape(y_pred.shape[0])
    print("accuracy =", accuracy_score(y_true, y_pred))

In [3]:
def reshape_input_sequence(data, shape):
    new_data = []
    for training_example in data:
        new_data.append(training_example[0:shape])
    new_data = np.array(new_data)
    return new_data

## 2) Criteria 1 - Angles

#### Loading training data

In [4]:
criteria1_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_1.npy", allow_pickle = True)

In [5]:
len(criteria1_data_angles)

657

#### Getting X_train, X_test, y_train, y_test

In [6]:
X = np.vstack(criteria1_data_angles[:, 0])
y = np.vstack(criteria1_data_angles[:, 1])
X_train_criteria1_angles, X_test_criteria1_angles, y_train_criteria1_angles, y_test_criteria1_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [7]:
print("X_train_criteria1_angles shape:", X_train_criteria1_angles.shape)
print("y_train_criteria1_angles shape:", y_train_criteria1_angles.shape)
print("X_test_criteria1_angles shape:", X_test_criteria1_angles.shape)
print("y_test_criteria1_angles shape:", y_test_criteria1_angles.shape)

X_train_criteria1_angles shape: (525, 600, 8)
y_train_criteria1_angles shape: (525, 1)
X_test_criteria1_angles shape: (132, 600, 8)
y_test_criteria1_angles shape: (132, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [8]:
new_length = 150
X_train_criteria1_angles = reshape_input_sequence(X_train_criteria1_angles, new_length)
X_test_criteria1_angles = reshape_input_sequence(X_test_criteria1_angles, new_length)
print("X_train_criteria1_angles shape:", X_train_criteria1_angles.shape)
print("y_train_criteria1_angles shape:", y_train_criteria1_angles.shape)
print("X_test_criteria1_angles shape:", X_test_criteria1_angles.shape)
print("y_test_criteria1_angles shape:", y_test_criteria1_angles.shape)

X_train_criteria1_angles shape: (525, 150, 8)
y_train_criteria1_angles shape: (525, 1)
X_test_criteria1_angles shape: (132, 150, 8)
y_test_criteria1_angles shape: (132, 1)


#### Building and training the model

In [167]:
criteria1_angles_model = Sequential(
    [
        GRU(units=265, input_shape=(X_train_criteria1_angles.shape[1], X_train_criteria1_angles.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], 
    name="criteria1_angles_model"
)

criteria1_angles_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria1_angles_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_48 (GRU)                    │ (None, 150, 265)       │       218,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 150, 265)       │         1,060 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_49 (GRU)                    │ (None, 150, 128)       │       151,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_50 (GRU)                    │ (None, 150, 64)        │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_51 (GRU)                    │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 418,950 (1.60 MB)

 Trainable params: 417,972 (1.59 MB)

 Non-trainable params: 978 (3.82 KB)

In [173]:
criteria1_angles_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [177]:
criteria1_angles_model.fit(X_train_criteria1_angles,y_train_criteria1_angles, epochs = 5, batch_size = 32)

Epoch 1/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 304ms/step - accuracy: 0.9965 - loss: 0.0240
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 305ms/step - accuracy: 1.0000 - loss: 0.0213
Epoch 3/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 303ms/step - accuracy: 0.9976 - loss: 0.0175
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 307ms/step - accuracy: 0.9996 - loss: 0.0160
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 300ms/step - accuracy: 0.9993 - loss: 0.0220


#### Saving the model

In [184]:
criteria1_angles_model.save("../models/other/bicep/angles/bicep_criteria1_angles_model_v2.h5")

#### Loading the model

In [10]:
criteria1_angles_model = tf.keras.models.load_model('../models/other/bicep/angles/bicep_criteria1_angles_model_v2.h5')

2024-04-14 13:05:00.911855: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-14 13:05:00.921906: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


#### Evaluating the model

###### Train Data

In [11]:
y_pred = criteria1_angles_model.predict(X_train_criteria1_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria1_angles)

17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step
accuracy = 0.9504761904761905


###### Test Data

In [12]:
y_pred = criteria1_angles_model.predict(X_test_criteria1_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria1_angles)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
accuracy = 0.9090909090909091


# -------------------------------------------------------------------------------------------------------------

## 3) Criteria 2 - Angles

#### Loading training data

In [13]:
criteria2_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_2.npy", allow_pickle = True)

In [14]:
len(criteria2_data_angles)

764

#### Getting X_train, X_test, y_train, y_test

In [15]:
X = np.vstack(criteria2_data_angles[:, 0])
y = np.vstack(criteria2_data_angles[:, 1])
X_train_criteria2_angles, X_test_criteria2_angles, y_train_criteria2_angles, y_test_criteria2_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [16]:
print("X_train_criteria2_angles shape:", X_train_criteria2_angles.shape)
print("y_train_criteria2_angles shape:", y_train_criteria2_angles.shape)
print("X_test_criteria2_angles shape:", X_test_criteria2_angles.shape)
print("y_test_criteria2_angles shape:", y_test_criteria2_angles.shape)

X_train_criteria2_angles shape: (611, 600, 8)
y_train_criteria2_angles shape: (611, 1)
X_test_criteria2_angles shape: (153, 600, 8)
y_test_criteria2_angles shape: (153, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [17]:
new_length = 150
X_train_criteria2_angles = reshape_input_sequence(X_train_criteria2_angles, new_length)
X_test_criteria2_angles = reshape_input_sequence(X_test_criteria2_angles, new_length)
print("X_train_criteria2_angles shape:", X_train_criteria2_angles.shape)
print("y_train_criteria2_angles shape:", y_train_criteria2_angles.shape)
print("X_test_criteria2_angles shape:", X_test_criteria2_angles.shape)
print("y_test_criteria2_angles shape:", y_test_criteria2_angles.shape)

X_train_criteria2_angles shape: (611, 150, 8)
y_train_criteria2_angles shape: (611, 1)
X_test_criteria2_angles shape: (153, 150, 8)
y_test_criteria2_angles shape: (153, 1)


#### Building and training the model

In [190]:
criteria2_angles_model = Sequential(
    [
        GRU(units=200, input_shape=(X_train_criteria2_angles.shape[1], X_train_criteria2_angles.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria2_angles_model"
)
criteria2_angles_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria2_angles_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_52 (GRU)                    │ (None, 150, 200)       │       126,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 150, 200)       │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_53 (GRU)                    │ (None, 150, 128)       │       126,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_53          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_54 (GRU)                    │ (None, 150, 128)       │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_55 (GRU)                    │ (None, 32)             │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_55          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 369,329 (1.41 MB)

 Trainable params: 368,353 (1.41 MB)

 Non-trainable params: 976 (3.81 KB)

In [200]:
criteria2_angles_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [202]:
criteria2_angles_model.fit(X_train_criteria2_angles,y_train_criteria2_angles, epochs = 10, batch_size = 32)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 302ms/step - accuracy: 0.9609 - loss: 0.0964
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 304ms/step - accuracy: 0.9856 - loss: 0.0489
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 297ms/step - accuracy: 0.9895 - loss: 0.0559
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 306ms/step - accuracy: 0.9820 - loss: 0.0691
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 307ms/step - accuracy: 0.9845 - loss: 0.0462
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 297ms/step - accuracy: 0.9823 - loss: 0.0548
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 315ms/step - accuracy: 0.9779 - loss: 0.0678
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 311ms/step - accuracy: 0.9845 - loss: 0.0596
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 312ms/step - accuracy: 0.9928 - loss: 0.0353
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 313ms/step - accuracy: 0.9776 - loss: 0.0479


#### Saving the model

In [206]:
criteria2_angles_model.save("../models/other/bicep/angles/bicep_criteria2_angles_model_v1.h5")

#### Loading the model

In [19]:
criteria2_angles_model = tf.keras.models.load_model('../models/other/bicep/angles/bicep_criteria2_angles_model_v1.h5')

#### Evaluating the model

###### Train Data

In [20]:
y_pred = criteria2_angles_model.predict(X_train_criteria2_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria2_angles)

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step
accuracy = 0.9541734860883797


###### Test Data

In [21]:
y_pred = criteria2_angles_model.predict(X_test_criteria2_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria2_angles)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step
accuracy = 0.9150326797385621


# -------------------------------------------------------------------------------------------------------------

## 4) Criteria 3 - Angles

#### Loading training data

In [22]:
criteria3_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_3.npy", allow_pickle = True)

In [23]:
len(criteria3_data_angles)

764

#### Getting X_train, X_test, y_train, y_test

In [24]:
X = np.vstack(criteria3_data_angles[:, 0])
y = np.vstack(criteria3_data_angles[:, 1])
X_train_criteria3_angles, X_test_criteria3_angles, y_train_criteria3_angles, y_test_criteria3_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [25]:
print("X_train_criteria3_angles shape:", X_train_criteria3_angles.shape)
print("y_train_criteria3_angles shape:", y_train_criteria3_angles.shape)
print("X_test_criteria3_angles shape:", X_test_criteria3_angles.shape)
print("y_test_criteria3_angles shape:", y_test_criteria3_angles.shape)

X_train_criteria3_angles shape: (611, 600, 8)
y_train_criteria3_angles shape: (611, 1)
X_test_criteria3_angles shape: (153, 600, 8)
y_test_criteria3_angles shape: (153, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [26]:
new_length = 150
X_train_criteria3_angles = reshape_input_sequence(X_train_criteria3_angles, new_length)
X_test_criteria3_angles = reshape_input_sequence(X_test_criteria3_angles, new_length)
print("X_train_criteria3_angles shape:", X_train_criteria3_angles.shape)
print("y_train_criteria3_angles shape:", y_train_criteria3_angles.shape)
print("X_test_criteria3_angles shape:", X_test_criteria3_angles.shape)
print("y_test_criteria3_angles shape:", y_test_criteria3_angles.shape)

X_train_criteria3_angles shape: (611, 150, 8)
y_train_criteria3_angles shape: (611, 1)
X_test_criteria3_angles shape: (153, 150, 8)
y_test_criteria3_angles shape: (153, 1)


#### Building and training the model

In [266]:
criteria3_angles_model = Sequential(
    [
        GRU(units=256, input_shape=(X_train_criteria3_angles.shape[1], X_train_criteria3_angles.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 150, return_sequences=True),
        BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 20),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria3_angles_model"
)
criteria3_angles_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria3_angles_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_74 (GRU)                    │ (None, 150, 256)       │       204,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_74          │ (None, 150, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_75 (GRU)                    │ (None, 150, 150)       │       183,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_75          │ (None, 150, 150)       │           600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_76 (GRU)                    │ (None, 150, 64)        │        41,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_76          │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_77 (GRU)                    │ (None, 20)             │         5,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_77          │ (None, 20)             │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 436,501 (1.67 MB)

 Trainable params: 435,521 (1.66 MB)

 Non-trainable params: 980 (3.83 KB)

In [317]:
criteria3_angles_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [318]:
criteria3_angles_model.fit(X_train_criteria3_angles, y_train_criteria3_angles, epochs = 10, batch_size = 32)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 296ms/step - accuracy: 0.9729 - loss: 0.0903
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 300ms/step - accuracy: 0.9811 - loss: 0.0718
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 305ms/step - accuracy: 0.9861 - loss: 0.0506
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 303ms/step - accuracy: 0.9928 - loss: 0.0437
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 296ms/step - accuracy: 0.9789 - loss: 0.0482
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 291ms/step - accuracy: 0.9912 - loss: 0.0389
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 294ms/step - accuracy: 0.9942 - loss: 0.0310
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 301ms/step - accuracy: 0.9847 - loss: 0.0455
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 299ms/step - accuracy: 0.9955 - loss: 0.0337
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 300ms/step - accuracy: 0.9960 - loss: 0.0259


#### Saving the model

In [331]:
criteria3_angles_model.save("../models/other/bicep/angles/bicep_criteria3_angles_model_v2.h5")

#### Loading the model

In [28]:
criteria3_angles_model = tf.keras.models.load_model('../models/other/bicep/angles/bicep_criteria3_angles_model_v2.h5')

#### Evaluating the model

###### Train Data

In [29]:
y_pred = criteria3_angles_model.predict(X_train_criteria3_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria3_angles)

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step
accuracy = 0.9967266775777414


###### Test Data

In [30]:
y_pred = criteria3_angles_model.predict(X_test_criteria3_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria3_angles)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step
accuracy = 0.9738562091503268


# -------------------------------------------------------------------------------------------------------------

## 5) Criteria 4 - Angles

#### Loading training data

In [39]:
criteria4_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_4.npy", allow_pickle = True)

In [40]:
len(criteria4_data_angles)

803

#### Getting X_train, X_test, y_train, y_test

In [41]:
X = np.vstack(criteria4_data_angles[:, 0])
y = np.vstack(criteria4_data_angles[:, 1])
X_train_criteria4_angles, X_test_criteria4_angles, y_train_criteria4_angles, y_test_criteria4_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [42]:
print("X_train_criteria4_angles shape:", X_train_criteria4_angles.shape)
print("y_train_criteria4_angles shape:", y_train_criteria4_angles.shape)
print("X_test_criteria4_angles shape:", X_test_criteria4_angles.shape)
print("y_test_criteria4_angles shape:", y_test_criteria4_angles.shape)

X_train_criteria4_angles shape: (642, 600, 8)
y_train_criteria4_angles shape: (642, 1)
X_test_criteria4_angles shape: (161, 600, 8)
y_test_criteria4_angles shape: (161, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [43]:
new_length = 150
X_train_criteria4_angles = reshape_input_sequence(X_train_criteria4_angles, new_length)
X_test_criteria4_angles = reshape_input_sequence(X_test_criteria4_angles, new_length)
print("X_train_criteria4_angles shape:", X_train_criteria4_angles.shape)
print("y_train_criteria4_angles shape:", y_train_criteria4_angles.shape)
print("X_test_criteria4_angles shape:", X_test_criteria4_angles.shape)
print("y_test_criteria4_angles shape:", y_test_criteria4_angles.shape)

X_train_criteria4_angles shape: (642, 150, 8)
y_train_criteria4_angles shape: (642, 1)
X_test_criteria4_angles shape: (161, 150, 8)
y_test_criteria4_angles shape: (161, 1)


#### Building and training the model

In [38]:
criteria4_angles_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria4_angles.shape[1], X_train_criteria4_angles.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria4_angles_model"
)
criteria4_angles_model.summary()

Model: "criteria4_angles_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 50)                11800     
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
criteria4_angles_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [40]:
criteria4_angles_model.fit(X_train_criteria4_angles, y_train_criteria4_angles, epochs = 10)

Epoch 1/10
21/21 [==============================] - 5s 191ms/step - loss: 0.6852 - accuracy: 0.5857
Epoch 2/10
21/21 [==============================] - 3s 168ms/step - loss: 0.6678 - accuracy: 0.6199
Epoch 3/10
21/21 [==============================] - 4s 180ms/step - loss: 0.6645 - accuracy: 0.6199
Epoch 4/10
21/21 [==============================] - 4s 195ms/step - loss: 0.6654 - accuracy: 0.6199
Epoch 5/10
21/21 [==============================] - 4s 210ms/step - loss: 0.6643 - accuracy: 0.6199
Epoch 6/10
21/21 [==============================] - 4s 203ms/step - loss: 0.6659 - accuracy: 0.6199
Epoch 7/10
21/21 [==============================] - 4s 190ms/step - loss: 0.6653 - accuracy: 0.6199
Epoch 8/10
21/21 [==============================] - 4s 202ms/step - loss: 0.6660 - accuracy: 0.6199
Epoch 9/10
21/21 [==============================] - 4s 190ms/step - loss: 0.6651 - accuracy: 0.6199
Epoch 10/10
21/21 [==============================] - 4s 186ms/step - loss: 0.6658 - accuracy: 0.6199

#### Saving the model

In [ ]:
criteria4_angles_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [41]:
y_pred = criteria4_angles_model.predict(X_train_criteria4_angles)
evaluate(y_pred, y_train_criteria4_angles)

21/21 [==============================] - 2s 71ms/step
accuracy = 0.6199376947040498


###### Test Data

In [42]:
y_pred = criteria4_angles_model.predict(X_test_criteria4_angles)
evaluate(y_pred, y_test_criteria4_angles)

6/6 [==============================] - 0s 62ms/step
accuracy = 0.6024844720496895


# -------------------------------------------------------------------------------------------------------------

## 6) Full Test

In [31]:
def test_models(input_sample):
    input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
    y_pred1 = criteria1_angles_model.predict(input_sample)
    y_pred1 = tf.nn.sigmoid(y_pred1)
    y_pred2 = criteria2_angles_model.predict(input_sample)
    y_pred2 = tf.nn.sigmoid(y_pred2)
    y_pred3 = criteria3_angles_model.predict(input_sample)
    y_pred3 = tf.nn.sigmoid(y_pred3)
    #y_pred4 = criteria4_angles_model.predict(input_sample)
    print("Criteria1:", y_pred1[0][0].numpy())
    print("Criteria2:", y_pred2[0][0].numpy())
    print("Criteria3:", y_pred3[0][0].numpy())
    #print("Criteria4:", y_pred4[0][0])

In [48]:
num = 11
input_sample = X_test_criteria4_angles[num]
print(y_test_criteria4_angles[num])
test_models(input_sample)
# results are good for criteria1_v2, criteria2_v1, criteria3_v2

[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Criteria1: 0.98517466
Criteria2: 0.6810883
Criteria3: 0.1648532


In [54]:
input_sample = X_train_criteria2_angles[3]
print(y_train_criteria2_angles[3])
test_models(input_sample)

[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Criteria1: 0.54988134
Criteria2: 0.007976919
Criteria3: 0.14794283


In [55]:
input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
y_pred1 = criteria1_angles_model.predict(input_sample)
y_pred1 = tf.nn.sigmoid(y_pred1)
y_pred1 = y_pred1[0][0].numpy()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


In [56]:
type(y_pred1)

numpy.float32

In [59]:
x = float(y_pred1)
type(x)

float

In [60]:
x

0.5498813390731812